## This project is about enriching bookshelf dataset

Goodreads API
https://www.goodreads.com/api

OpenLibrary API
https://openlibrary.org/dev/docs/api/books

Lets read the dataset

In [122]:
!wget -O MyBooks.csv https://www.dropbox.com/s/fk98zlrx4lypz04/MyBooks.csv?dl=0

--2020-07-09 00:22:15--  https://www.dropbox.com/s/fk98zlrx4lypz04/MyBooks.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fk98zlrx4lypz04/MyBooks.csv [following]
--2020-07-09 00:22:15--  https://www.dropbox.com/s/raw/fk98zlrx4lypz04/MyBooks.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc135fc2c14cdf8692a403ce35bb.dl.dropboxusercontent.com/cd/0/inline/A7IQJiUgnd1kOMN3wQk5sl7qQc2zw5kM69Kdh3eBrvKlGtWR9RmyVYgRe0EOuL8JR3h6u8NLaZGuRdAXO54CmoeYUbsZrSqa_-zSjjKCw0HPJfqgsi6McGXHT1P31ti1VnM/file# [following]
--2020-07-09 00:22:15--  https://uc135fc2c14cdf8692a403ce35bb.dl.dropboxusercontent.com/cd/0/inline/A7IQJiUgnd1kOMN3wQk5sl7qQc2zw5kM69Kdh3eBrvKlGtWR9RmyVYgRe0EOuL8JR3h6u8NLaZGuRdAXO54CmoeYUbsZrSqa_-zSjjKCw0HPJfqg

In [123]:
# install some packages
!pip install rauth

print("done..")

done..


In [207]:
# import some libs
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import rauth
import requests
import json

from rauth.service import OAuth1Service, OAuth1Session

print("done..")

done..


In [262]:
# inirial dataset

df = pd.read_csv("MyBooks.csv", error_bad_lines=False)
df.head()

,ISBN Code,Title,Author,Published Date,Memo,Page count,Language,Country,Desciption,Image(URL)
0,9789663437354,,,,,,,,,NaN
1,9789664620397,,,,,,,,,NaN
2,9786176951223,,,,,,,,,NaN
3,9786177084180,Сказки для малышей,,2016,,61,ru,US,,NaN
4,9789664625453,,,,,,,,,NaN


### Goodreads enrishment attempt

In [266]:
# lets define sone useful stuff
# https://www.goodreads.com/api/keys
CONSUMER_KEY = "__GO_GET_YOUR_OWN_KEY__"
CONSUMER_SECRET = "__GO_GET_YOUR_OWN_SECRET__"

goodreads = OAuth1Service(
    consumer_key = CONSUMER_KEY,
    consumer_secret = CONSUMER_SECRET,
    name = 'goodreads',
    request_token_url = 'https://www.goodreads.com/oauth/request_token',
    authorize_url = 'https://www.goodreads.com/oauth/authorize',
    access_token_url = 'https://www.goodreads.com/oauth/access_token',
    base_url = 'https://www.goodreads.com/'
    )

# head_auth=True is important here; this doesn't work with oauth2 for some reason
request_token, request_token_secret = goodreads.get_request_token(header_auth=True)
authorize_url = goodreads.get_authorize_url(request_token)
print("Visit this URL in your browser: ", authorize_url)

Visit this URL in your browser:  https://www.goodreads.com/oauth/authorize?oauth_token=sLXU7xyBb6YwwZNSmaQTjg


In [ ]:
session = goodreads.get_auth_session(request_token, request_token_secret)
request_url = "https://www.goodreads.com/book/isbn/{}?key={}"

In [275]:
count = 0

EMPTY = "    Empty"
NOT_EMPTY = "Not Empty"
FOUND = "Found"
NOT_FOUND = "Not Found"

for index, row in df.iterrows():
    responce = session.get(request_url.format(row["ISBN Code "], CONSUMER_KEY))
    root = ET.fromstring(responce.content)

    old_title = EMPTY if len(row["Title "]) == 1 else NOT_EMPTY
    
    try:
        new_title = str(root[1][1].text)
        print(old_title, " => ", FOUND)
        if old_title != NOT_EMPTY:
            count += 1 
    except: 
        print(old_title, " => ", NOT_FOUND)
        pass

print("Ok, we found {} useful updates in goodreads database".format(count))

    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
Not Empty  =>  Not Found
    Empty  =>  Not Found
Not Empty  =>  Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Found
    Empty  =>  Found
    Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Not Found
Not Empty  =>  Found
    Empty  =>  Not Found
    Empty  =>  Not Found
Not Empty  =>  Found
Not Empty  =>  Found
Not Empty  =>  Found
    Empty  =>  Not Found
    Empty  =>  Found
Not Empty  =>  Not Found
    Empty  =>  Not Found
    Empty  =>  Found
Not Empty  =>  Found
Not Empty  =>  Found
    Empty  =>  Not Found
Ok, we found 4 useful updates in goodreads database


### OpenLibrary enrishment attempt

In [260]:
openlibraryapi = "https://openlibrary.org/api/books?bibkeys=ISBN:{}"

for index, row in df.iterrows():
    isbn = row["ISBN Code "]
    resp = requests.get(openlibraryapi.format(isbn))
    payload = resp.text.replace("var _OLBookInfo = {", "{").replace("};", "}")
    if payload != "{}":
        parsed_payload = json.loads(payload)
        url = parsed_payload["ISBN:{}".format(row["ISBN Code "])]["preview_url"]
        print("we found potential update for ISBN {}, here are more details: {}".format(isbn, url))
    else:
        print("no updates found for ISBN {}".format(isbn))
        

no updates found for ISBN 9789663437354
no updates found for ISBN 9789664620397
no updates found for ISBN 9786176951223
no updates found for ISBN 9786177084180
no updates found for ISBN 9789664625453
no updates found for ISBN 9789667047856
no updates found for ISBN 966716134X 
no updates found for ISBN 9789661200394
no updates found for ISBN 9789663190792
no updates found for ISBN 9663190795
no updates found for ISBN 9789661800907
no updates found for ISBN 9786176951889
no updates found for ISBN 9789662522082
no updates found for ISBN 9789661694629
no updates found for ISBN 9786176903109
we found potential update for ISBN 9789667047818, here are more details: https://openlibrary.org/books/OL27644524M/Povitrjani_ribki
no updates found for ISBN 9789664623473
no updates found for ISBN 9786177084692
no updates found for ISBN 9789669761057
no updates found for ISBN 9786175850275
no updates found for ISBN 9789662522075
no updates found for ISBN 9789669172075
no updates found for ISBN 9789664

In [ ]:
# debug for GoodReads API
root = ET.fromstring(responce.content)
tag_indexes = [1,7,8,9,10,13,15,18,19,20,21,22,23,24]
for index in tag_indexes:
    print(root[1][index].tag + ": " + str(root[1][index].text))